<a href="https://colab.research.google.com/github/imnotwannafire/Head_dectection_model_with_quantization/blob/main/B%E1%BA%A3n_sao_c%E1%BB%A7a_training_custom_model_for_head_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ultralytics
!pip install roboflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 68.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 108.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

In [2]:
from IPython import display
display.clear_output()
!yolo mode=checks

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Traceback (most recent call last):
  File "/usr/local/bin/yolo", line 8, in <module>
    sys.exit(entrypoint())
             ^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/ultralytics/cfg/__init__.py", line 918, in entrypoint
    raise ValueError(f"Invalid 'mode={mode}'. Valid modes are {list(MODES)}.\n{CLI_HELP_MSG}")
ValueError: Invalid 'mode=<module 'ultralytics.utils.checks' from '/usr/local/lib/python3.11/dist-packages/ultralytics/utils/checks.py'>'. Valid modes are ['export', 'benchmark', 'predict', 'track', 'train', 'val'].

    Arguments received: ['yolo', 'mode=checks']. Ultralytics 'yolo' commands use the following syntax:

        yolo TASK MODE ARGS

    

In [3]:
from google.colab import drive
import torch
import os

# Mount Drive và cài đặt
drive.mount('/content/drive')

from ultralytics import YOLO
from roboflow import Roboflow

# Kiểm tra GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    torch.cuda.empty_cache()

MessageError: Error: credential propagation was unsuccessful

In [ ]:
%mkdir /content/drive/MyDrive/grove_ai_qat_training

In [ ]:
%mkdir /content/drive/MyDrive/grove_ai_qat_training/dataset/

In [ ]:
%cd /content/drive/MyDrive/grove_ai_qat_training/dataset/

/content/drive/MyDrive/grove_ai_qat_training/dataset


In [ ]:
%cd /content/

/content


In [4]:
from roboflow import Roboflow
rf = Roboflow(api_key="xEK6F5JlUTrbdECVPjxE")
project = rf.workspace("grovevision").project("head_detection_on_bus")
version = project.version(4)
dataset = version.download("yolov8")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to head_detection_on_bus-4 in yolov8:: 100%|██████████| 17742/17742 [00:02<00:00, 6484.17it/s]


In [5]:
%cd /content/drive/MyDrive/grove_ai_qat_training/

[Errno 2] No such file or directory: '/content/drive/MyDrive/grove_ai_qat_training/'
/content


In [6]:
from ultralytics import YOLO
import os
import torch
import numpy as np
import yaml
import time
import shutil
from pathlib import Path
import cv2

# Clear GPU cache và setup
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print(f"✅ GPU: {torch.cuda.get_device_name(0)}")
    print(f"✅ GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")

# ===== CONFIGURATION =====
DATA_YAML_PATH = '/content/head_detection_on_bus-4/data.yaml'
WORKSPACE = '/content/grove_ai_qat_training'
os.makedirs(WORKSPACE, exist_ok=True)

print(f"📁 Workspace: {WORKSPACE}")
print(f"📊 Dataset: {DATA_YAML_PATH}")

# Verify dataset exists
if not os.path.exists(DATA_YAML_PATH):
    raise FileNotFoundError(f"Dataset not found: {DATA_YAML_PATH}")

✅ GPU: Tesla T4
✅ GPU Memory: 14.7 GB
📁 Workspace: /content/grove_ai_qat_training
📊 Dataset: /content/head_detection_on_bus-4/data.yaml


In [ ]:
print("🚀 STAGE 1: HIGH RESOLUTION TRAINING (640x640)")
print("="*60)

# Initialize YOLOv8n model
model_stage1 = YOLO('yolov8n.pt')

# Stage 1 Configuration
stage1_config = {
    'data': DATA_YAML_PATH,
    'imgsz': 640,                     # High resolution cho feature learning
    'epochs': 60,
    'batch': 16,                      # Optimized cho 15GB GPU
    'name': 'stage1_640_progressive',
    'project': WORKSPACE,
    'patience': 25,
    'optimizer': 'AdamW',
    'lr0': 0.01,                      # Standard learning rate
    'device': 0,
    'augment': True,
    'save_period': 10,
    'amp': True,                      # Mixed precision
    'workers': 8,
    'cache': True,
    'cos_lr': True,
    'warmup_epochs': 3
}

print("📋 Stage 1 Configuration:")
for key, value in stage1_config.items():
    if key not in ['data', 'project']:
        print(f"   {key}: {value}")

# Execute Stage 1 training
start_time = time.time()
results_stage1 = model_stage1.train(**stage1_config)
stage1_duration = (time.time() - start_time) / 3600



🚀 STAGE 1: HIGH RESOLUTION TRAINING (640x640)
📋 Stage 1 Configuration:
   imgsz: 640
   epochs: 60
   batch: 16
   name: stage1_640_progressive
   patience: 25
   optimizer: AdamW
   lr0: 0.01
   device: 0
   augment: True
   save_period: 10
   amp: True
   workers: 8
   cache: True
   cos_lr: True
   warmup_epochs: 3
Ultralytics 8.3.157 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/content/drive/MyDrive/grove_ai_qat_training/head_detection_on_bus-4/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=60, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, i

100%|██████████| 755k/755k [00:00<00:00, 23.9MB/s]

Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

Model summary: 129 layers, 3,011,043 parameters, 3,011,027 gradients, 8.2 GFLOPs

Transferred 319/355 items from pretrained weights
Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...


100%|██████████| 5.35M/5.35M [00:00<00:00, 83.0MB/s]


...

                   all        479        889      0.961      0.948      0.982      0.748



60 epochs completed in 2.219 hours.
Optimizer stripped from /content/drive/MyDrive/grove_ai_qat_training/stage1_640_progressive/weights/last.pt, 6.2MB
Optimizer stripped from /content/drive/MyDrive/grove_ai_qat_training/stage1_640_progressive/weights/best.pt, 6.2MB

Validating /content/drive/MyDrive/grove_ai_qat_training/stage1_640_progressive/weights/best.pt...
Ultralytics 8.3.157 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 15/15 [00:04<00:00,  3.17it/s]


                   all        479        889      0.946      0.958      0.982       0.74
Speed: 0.2ms preprocess, 4.3ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to /content/drive/MyDrive/grove_ai_qat_training/stage1_640_progressive


AttributeError: 'DetMetrics' object has no attribute 'save_dir'. See valid attributes below.

    Utility class for computing detection metrics such as precision, recall, and mean average precision (mAP).

    Attributes:
        names (Dict[int, str]): A dictionary of class names.
        box (Metric): An instance of the Metric class for storing detection results.
        speed (Dict[str, float]): A dictionary for storing execution times of different parts of the detection process.
        task (str): The task type, set to 'detect'.
        stats (Dict[str, List]): A dictionary containing lists for true positives, confidence scores, predicted classes, target classes, and target images.
        nt_per_class: Number of targets per class.
        nt_per_image: Number of targets per image.
    

In [ ]:
# Get Stage 1 model path
stage1_model_path = '/content/grove_ai_qat_training/stage1_best.pt'
model_stage1 = YOLO(stage1_model_path)
if os.path.exists(stage1_model_path):
    stage1_size = os.path.getsize(stage1_model_path) / (1024**2)
    # print(f"✅ Stage 1 completed in {stage1_duration:.2f}h")
    print(f"   Model: {stage1_model_path}")
    print(f"   Size: {stage1_size:.2f}MB")

    # Backup to workspace
    shutil.copy2(stage1_model_path, os.path.join(WORKSPACE, 'stage1_best.pt'))

    # Quick validation
    val_results_s1 = model_stage1.val(data=DATA_YAML_PATH, imgsz=640, verbose=False)
    print(f"   mAP@0.5: {val_results_s1.box.map50:.4f}")
else:
    raise FileNotFoundError("Stage 1 training failed!")
# stage1_model_path = '/content/drive/MyDrive/grove_ai_qat_training/stage1_640_progressive/weights/best.pt'
# print(stage1_model_path)

   Model: /content/grove_ai_qat_training/stage1_best.pt
   Size: 5.95MB


SameFileError: '/content/grove_ai_qat_training/stage1_best.pt' and '/content/grove_ai_qat_training/stage1_best.pt' are the same file

In [ ]:
print("\n🎯 STAGE 2: FINE-TUNING (192x192)")
print("="*50)

# Load Stage 1 model
model_stage2 = YOLO(stage1_model_path)
print(f"✅ Loaded Stage 1 model for fine-tuning")

# Stage 2 Configuration
stage2_config = {
    'data': DATA_YAML_PATH,           # 🔥 SAME dataset
    'imgsz': 192,                     # Target resolution cho Grove AI
    'epochs': 40,
    'batch': 32,                      # Higher batch với lower resolution
    'name': 'stage2_192_finetune',
    'project': WORKSPACE,
    'patience': 20,
    'optimizer': 'AdamW',
    'lr0': 0.001,                     # 10x lower than Stage 1
    'lrf': 0.0001,
    'warmup_epochs': 5,
    'warmup_momentum': 0.8,
    'cos_lr': True,
    'augment': True,
    'save_period': 5,
    'amp': True,
    'workers': 8
}

print("📋 Stage 2 Configuration:")
print(f"   Target resolution: {stage2_config['imgsz']}x{stage2_config['imgsz']}")
print(f"   Learning rate: {stage2_config['lr0']} (fine-tuning)")
print(f"   Batch size: {stage2_config['batch']}")

# Execute Stage 2 training
start_time = time.time()
results_stage2 = model_stage2.train(**stage2_config)
stage2_duration = (time.time() - start_time) / 60


🎯 STAGE 2: FINE-TUNING (192x192)
✅ Loaded Stage 1 model for fine-tuning
📋 Stage 2 Configuration:
   Target resolution: 192x192
   Learning rate: 0.001 (fine-tuning)
   Batch size: 32
Ultralytics 8.3.157 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/content/head_detection_on_bus-4/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=40, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=192, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.0001, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=/content/grove_ai_

100%|██████████| 755k/755k [00:00<00:00, 27.3MB/s]


                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128

Model summary: 129 layers, 3,011,043 parameters, 3,011,027 gradients, 8.2 GFLOPs

Transferred 355/355 items from pretrained weights
Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...


100%|██████████| 5.35M/5.35M [00:00<00:00, 107MB/s]


...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      40/40      1.13G      1.196      0.604     0.9566         27        192: 100%|██████████| 261/261 [00:40<00:00,  6.52it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:01<00:00,  4.17it/s]


                   all        479        889      0.914      0.841      0.923       0.58

40 epochs completed in 0.487 hours.
Optimizer stripped from /content/grove_ai_qat_training/stage2_192_finetune/weights/last.pt, 6.2MB
Optimizer stripped from /content/grove_ai_qat_training/stage2_192_finetune/weights/best.pt, 6.2MB

Validating /content/grove_ai_qat_training/stage2_192_finetune/weights/best.pt...
Ultralytics 8.3.157 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:03<00:00,  2.55it/s]


                   all        479        889      0.883      0.787       0.89       0.56
Speed: 0.1ms preprocess, 2.0ms inference, 0.0ms loss, 1.9ms postprocess per image
Results saved to /content/grove_ai_qat_training/stage2_192_finetune


In [ ]:
# Get Stage 2 model path
stage2_model_path = os.path.join('/content/grove_ai_qat_training/stage2_192_finetune', 'weights', 'best.pt')

if os.path.exists(stage2_model_path):
    stage2_size = os.path.getsize(stage2_model_path) / (1024**2)
    print(f"✅ Stage 2 completed in {stage2_duration:.1f}min")
    print(f"   Model: {stage2_model_path}")
    print(f"   Size: {stage2_size:.2f}MB")

    # Backup to workspace
    shutil.copy2(stage2_model_path, os.path.join(WORKSPACE, 'stage2_best.pt'))

    # Validation
    val_results_s2 = model_stage2.val(data=DATA_YAML_PATH, imgsz=192, verbose=False)
    print(f"   mAP@0.5: {val_results_s2.box.map50:.4f}")
else:
    raise FileNotFoundError("Stage 2 training failed!")

✅ Stage 2 completed in 29.7min
   Model: /content/grove_ai_qat_training/stage2_192_finetune/weights/best.pt
   Size: 5.91MB
Ultralytics 8.3.157 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 780.1±214.1 MB/s, size: 30.5 KB)


val: Scanning /content/head_detection_on_bus-4/valid/labels.cache... 479 images, 0 backgrounds, 0 corrupt: 100%|██████████| 479/479 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 15/15 [00:03<00:00,  4.60it/s]


                   all        479        889      0.886      0.787      0.891      0.561
Speed: 0.1ms preprocess, 2.8ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to /content/grove_ai_qat_training/stage2_192_finetune2
   mAP@0.5: 0.8912


In [ ]:
print("\n💎 STAGE 3: QUANTIZATION-AWARE TRAINING (QAT)")
print("="*55)

# Load Stage 2 model
model_qat = YOLO(stage2_model_path)
print(f"✅ Loaded Stage 2 model for QAT")

# QAT Configuration - CRITICAL PARAMETERS
qat_config = {
    'data': DATA_YAML_PATH,
    'imgsz': 192,                     # Grove AI Vision target
    'epochs': 25,                     # QAT needs fewer epochs
    'batch': 48,                      # Slightly reduced for stability
    'name': 'qat_grove_ai_optimized',
    'project': WORKSPACE,
    'patience': 12,
    'optimizer': 'AdamW',

    # 🔥 CRITICAL: Ultra-low learning rates cho QAT
    'lr0': 0.00005,                   # 200x lower than Stage 1
    'lrf': 0.000005,                  # Extremely low final LR
    'momentum': 0.937,
    'weight_decay': 0.0005,
    'warmup_epochs': 2,
    'warmup_momentum': 0.8,

    # Minimal augmentation cho QAT stability
    'degrees': 1.0,                   # Minimal rotation
    'translate': 0.01,                # Minimal translation
    'scale': 0.01,                    # Minimal scaling
    'shear': 0.2,                     # Minimal shear
    'perspective': 0.000005,          # Almost no perspective
    'fliplr': 0.5,                    # Keep horizontal flip

    # Disable complex augmentations
    'mixup': 0.0,
    'mosaic': 0.0,
    'copy_paste': 0.0,
    'hsv_h': 0.005,
    'hsv_s': 0.3,
    'hsv_v': 0.2,

    # QAT-specific settings
    'cos_lr': True,
    'amp': False,                     # 🔥 CRITICAL: No mixed precision
    'save_period': 3,
    'val': True,
    'plots': True,
    'workers': 6,
    'cache': True
}

print("📋 QAT Configuration:")
print(f"   Learning rate: {qat_config['lr0']} (ultra-low)")
print(f"   Augmentation: Minimal (stability focus)")
print(f"   Mixed precision: Disabled")
print(f"   Target: INT8 quantization ready")

# Clear GPU cache
if torch.cuda.is_available():
    torch.cuda.empty_cache()

# Execute QAT training
print("🔄 Starting QAT training...")
start_time = time.time()
results_qat = model_qat.train(**qat_config)
qat_duration = (time.time() - start_time) / 60




💎 STAGE 3: QUANTIZATION-AWARE TRAINING (QAT)
✅ Loaded Stage 2 model for QAT
📋 QAT Configuration:
   Learning rate: 5e-05 (ultra-low)
   Augmentation: Minimal (stability focus)
   Mixed precision: Disabled
   Target: INT8 quantization ready
🔄 Starting QAT training...
Ultralytics 8.3.157 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=False, augment=False, auto_augment=randaugment, batch=48, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/content/head_detection_on_bus-4/data.yaml, degrees=1.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=25, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.005, hsv_s=0.3, hsv_v=0.2, imgsz=192, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=5e-05, l

train: Scanning /content/head_detection_on_bus-4/train/labels.cache... 8331 images, 3 backgrounds, 0 corrupt: 100%|██████████| 8331/8331 [00:00<?, ?it/s]

WARNING ⚠️ cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.



train: Caching images (0.5GB RAM): 100%|██████████| 8331/8331 [00:13<00:00, 619.36it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


...

                   all        479        889      0.939      0.828      0.926      0.588



25 epochs completed in 0.203 hours.
Optimizer stripped from /content/grove_ai_qat_training/qat_grove_ai_optimized/weights/last.pt, 6.2MB
Optimizer stripped from /content/grove_ai_qat_training/qat_grove_ai_optimized/weights/best.pt, 6.2MB

Validating /content/grove_ai_qat_training/qat_grove_ai_optimized/weights/best.pt...
Ultralytics 8.3.157 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:02<00:00,  1.75it/s]


                   all        479        889      0.939      0.829      0.927      0.589
Speed: 0.0ms preprocess, 0.5ms inference, 0.0ms loss, 2.5ms postprocess per image
Results saved to /content/grove_ai_qat_training/qat_grove_ai_optimized


AttributeError: 'DetMetrics' object has no attribute 'save_dir'. See valid attributes below.

    Utility class for computing detection metrics such as precision, recall, and mean average precision (mAP).

    Attributes:
        names (Dict[int, str]): A dictionary of class names.
        box (Metric): An instance of the Metric class for storing detection results.
        speed (Dict[str, float]): A dictionary for storing execution times of different parts of the detection process.
        task (str): The task type, set to 'detect'.
        stats (Dict[str, List]): A dictionary containing lists for true positives, confidence scores, predicted classes, target classes, and target images.
        nt_per_class: Number of targets per class.
        nt_per_image: Number of targets per image.
    

In [10]:
# Get QAT model path
qat_model_path = os.path.join('/content/grove_ai_qat_training/qat_grove_ai_optimized', 'weights', 'best.pt')
print(qat_model_path)
if os.path.exists(qat_model_path):
    qat_size = os.path.getsize(qat_model_path) / (1024**2)
    print(f"✅ QAT completed in {qat_duration:.1f}min")
    print(f"   Model: {qat_model_path}")
    print(f"   Size: {qat_size:.2f}MB")

    # Backup to workspace
    shutil.copy2(qat_model_path, os.path.join(WORKSPACE, 'qat_best.pt'))

    # Validation
    val_results_qat = model_qat.val(data=DATA_YAML_PATH, imgsz=192, verbose=False)
    print(f"   mAP@0.5: {val_results_qat.box.map50:.4f}")
else:
    raise FileNotFoundError("QAT training failed!")

/content/grove_ai_qat_training/qat_grove_ai_optimized/weights/best.pt


NameError: name 'qat_duration' is not defined

In [24]:
import os
import yaml
import numpy as np
import cv2
import glob
from pathlib import Path

def create_representative_dataset(data_yaml_path, num_samples=100, split="val"):
    """
    Tạo representative dataset cho optimal TFLite calibration

    Args:
        data_yaml_path: Path tới file data.yaml
        num_samples: Số lượng images để sample (default: 100)
        split: Dataset split để sử dụng ("val", "train", "test")

    Returns:
        Generator function cho TFLite calibration hoặc None nếu failed
    """

    print(f"📊 Creating representative dataset from {split} set...")

    try:
        # Get dataset base directory (thư mục chứa data.yaml)
        dataset_base_dir = Path(data_yaml_path).parent
        print(f"📂 Dataset base directory: {dataset_base_dir}")

        # Load YAML configuration
        with open(data_yaml_path, 'r') as f:
            data_config = yaml.safe_load(f)

        # Display dataset info
        print(f"📋 Dataset info:")
        print(f"   Classes: {data_config.get('nc', 'Unknown')} - {data_config.get('names', [])}")
        if 'roboflow' in data_config:
            print(f"   Project: {data_config['roboflow'].get('project', 'Unknown')}")

        # Get image path for specified split
        available_splits = [key for key in ['val', 'valid', 'train', 'test'] if key in data_config]
        print(f"   Available splits: {available_splits}")

        if split not in data_config:
            print(f"❌ Split '{split}' not found in data.yaml")
            print(f"   Available splits: {available_splits}")
            return None

        # Resolve relative path to absolute path
        # relative_path = data_config[split]
        relative_path = data_config[split].lstrip('/')
        images_dir = (dataset_base_dir / relative_path).resolve()

        print(f"📍 Image path resolution:")
        print(f"   Relative: {relative_path}")
        print(f"   Absolute: {images_dir}")

        # Verify directory exists
        if not images_dir.is_dir():
            print(f"❌ Image directory not found: {images_dir}")
            return None

        # Find image files
        image_extensions = ['*.jpg', '*.jpeg', '*.png', '*.bmp', '*.tiff']
        image_files = []

        for ext in image_extensions:
            # Search for both lowercase and uppercase extensions
            pattern_lower = images_dir / ext
            pattern_upper = images_dir / ext.upper()

            image_files.extend(glob.glob(str(pattern_lower)))
            image_files.extend(glob.glob(str(pattern_upper)))

        # Remove duplicates (case-insensitive systems might return duplicates)
        image_files = list(set(image_files))

        if not image_files:
            print(f"❌ No images found in {images_dir}")
            # Show directory contents for debugging
            try:
                contents = list(images_dir.iterdir())[:10]
                print(f"   Directory contents (first 10): {[f.name for f in contents]}")
            except:
                print("   Cannot list directory contents")
            return None

        # Sample representative images
        np.random.seed(42)  # Reproducible sampling
        np.random.shuffle(image_files)
        selected_files = image_files[:min(num_samples, len(image_files))]

        print(f"✅ Representative dataset created:")
        print(f"   Total images found: {len(image_files)}")
        print(f"   Selected for calibration: {len(selected_files)}")
        print(f"   Sample files: {[Path(f).name for f in selected_files[:3]]}...")

        def representative_data_gen():
            """Generator function cho TFLite calibration"""
            successful_count = 0

            for img_path in selected_files:
                try:
                    # Load image
                    img = cv2.imread(img_path)
                    if img is None:
                        continue

                    # Letterbox resize to 192x192 (Grove AI Vision format)
                    h, w = img.shape[:2]
                    r = min(192/h, 192/w)
                    new_w, new_h = int(w * r), int(h * r)

                    # Resize image
                    resized = cv2.resize(img, (new_w, new_h), interpolation=cv2.INTER_LINEAR)

                    # Create canvas với gray padding (114) - YOLOv8 standard
                    canvas = np.full((192, 192, 3), 114, dtype=np.uint8)
                    y_offset = (192 - new_h) // 2
                    x_offset = (192 - new_w) // 2
                    canvas[y_offset:y_offset+new_h, x_offset:x_offset+new_w] = resized

                    # Normalize to [0,1] và add batch dimension
                    processed = canvas.astype(np.float32) / 255.0
                    processed = np.expand_dims(processed, axis=0)

                    successful_count += 1
                    yield [processed]

                except Exception as e:
                    continue

            print(f"✅ Successfully processed {successful_count}/{len(selected_files)} images for calibration")

        return representative_data_gen

    except Exception as e:
        print(f"❌ Error creating representative dataset: {e}")
        import traceback
        traceback.print_exc()
        return None

In [25]:
print("\n📦 EXPORTING TO TFLITE (INT8 QUANTIZED)")
print("="*50)

# Load QAT model
model_export = YOLO(qat_model_path)

# Create representative dataset
rep_dataset_gen = create_representative_dataset(DATA_YAML_PATH)


📦 EXPORTING TO TFLITE (INT8 QUANTIZED)
📊 Creating representative dataset from val set...
📂 Dataset base directory: /content/head_detection_on_bus-4
📋 Dataset info:
   Classes: 1 - ['head']
   Project: head_detection_on_bus
   Available splits: ['val', 'train', 'test']
📍 Image path resolution:
   Relative: valid/images
   Absolute: /content/head_detection_on_bus-4/valid/images
✅ Representative dataset created:
   Total images found: 479
   Selected for calibration: 100
   Sample files: ['img_0155_jpg.rf.cd80fb1e2f9ce115e7f7ddcb840bc89e.jpg', '4EcUDt3baX_jpeg.rf.f1126c8a335a9328699d3727c1339ae2.jpg', 'img_1372_jpg.rf.7cdbd9bca01d4f64669cf03803bf5404.jpg']...


In [27]:
if rep_dataset_gen is None:
    print("⚠️  Exporting without representative dataset calibration...")

    # Basic export
    tflite_path = model_export.export(
        format='tflite',
        imgsz=192,
        int8=True,
        optimize=True,
        simplify=True,
        dynamic=False,
        batch=1
    )

else:
    print("🎯 Exporting với representative dataset calibration...")

    try:
        # Export to SavedModel first
        print("   Step 1: Exporting to SavedModel...")
        saved_model_path = model_export.export(
            format='saved_model',
            imgsz=192,
            optimize=True,
            simplify=True,
            dynamic=False,
            batch=1
        )

        # Manual TFLite conversion với calibration
        print("   Step 2: Converting to TFLite với calibration...")
        import tensorflow as tf

        converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_path)
        converter.optimizations = [tf.lite.Optimize.DEFAULT]
        converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
        converter.inference_input_type = tf.uint8
        converter.inference_output_type = tf.uint8
        converter.representative_dataset = rep_dataset_gen

        # Convert
        tflite_model = converter.convert()

        # Save calibrated TFLite
        tflite_path = os.path.join(WORKSPACE, 'grove_ai_optimized_int8.tflite')
        with open(tflite_path, 'wb') as f:
            f.write(tflite_model)

        print(f"✅ Calibrated TFLite saved: {tflite_path}")

    except Exception as e:
        print(f"⚠️  Calibration failed: {e}")
        print("   Falling back to basic export...")

        # Fallback to basic export
        tflite_path = model_export.export(
            format='tflite',
            imgsz=192,
            int8=True,
            optimize=True,
            simplify=True
        )

# Verify exported model
if os.path.exists(tflite_path):
    tflite_size = os.path.getsize(tflite_path) / (1024**2)
    print(f"📦 TFLite Model Information:")
    print(f"   File: {os.path.basename(tflite_path)}")
    print(f"   Size: {tflite_size:.2f}MB")
    print(f"   Target: Grove AI Vision (ARM Cortex-M55 + Ethos-U55)")
    print(f"   Input: 192x192x3 (INT8)")

    # Copy to final location
    final_tflite = os.path.join(WORKSPACE, 'grove_ai_final_deployment.tflite')
    shutil.copy2(tflite_path, final_tflite)
    print(f"✅ Final deployment model: {final_tflite}")
else:
    raise FileNotFoundError("TFLite export failed")

🎯 Exporting với representative dataset calibration...
   Step 1: Exporting to SavedModel...
Ultralytics 8.3.158 🚀 Python-3.11.13 torch-2.6.0+cu124 CPU (Intel Xeon 2.00GHz)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs

PyTorch: starting from '/content/grove_ai_qat_training/qat_grove_ai_optimized/weights/best.pt' with input shape (1, 3, 192, 192) BCHW and output shape(s) (1, 5, 756) (5.9 MB)

TensorFlow SavedModel: starting export with tensorflow 2.18.0...

ONNX: starting export with onnx 1.17.0 opset 19...
ONNX: slimming with onnxslim 0.1.58...
ONNX: export success ✅ 1.5s, saved as '/content/grove_ai_qat_training/qat_grove_ai_optimized/weights/best.onnx' (11.5 MB)
TensorFlow SavedModel: starting TFLite export with onnx2tf 1.28.0...
Saved artifact at '/content/grove_ai_qat_training/qat_grove_ai_optimized/weights/best_saved_model'. The following endpoints are available:

* Endpoint 'serving_default'
  inputs_0 (POSITIONAL_ONLY): TensorSpec(shape=(1, 192,

In [ ]:
print("\n📊 COMPREHENSIVE PIPELINE VALIDATION")
print("="*60)

# Performance comparison
models_info = {
    'Stage 1 (640→192)': (YOLO(stage1_model_path), 192),
    'Stage 2 (192)': (YOLO(stage2_model_path), 192),
    'QAT (192)': (YOLO(qat_model_path), 192)
}

results = {}
for name, (model, imgsz) in models_info.items():
    print(f"\n🔍 Validating {name}...")
    val_result = model.val(data=DATA_YAML_PATH, imgsz=imgsz, verbose=False)

    results[name] = {
        'map50': val_result.box.map50,
        'map50_95': val_result.box.map,
        'precision': val_result.box.mp,
        'recall': val_result.box.mr
    }

    print(f"   mAP@0.5: {val_result.box.map50:.4f}")

# TFLite inference speed test
print(f"\n⚡ TESTING TFLITE INFERENCE SPEED")
try:
    import tensorflow as tf

    interpreter = tf.lite.Interpreter(model_path=tflite_path)
    interpreter.allocate_tensors()

    input_details = interpreter.get_input_details()
    dummy_input = np.random.randint(0, 255, input_details[0]['shape'], dtype=input_details[0]['dtype'])

    # Warmup
    for _ in range(10):
        interpreter.set_tensor(input_details[0]['index'], dummy_input)
        interpreter.invoke()

    # Measure speed
    start_time = time.time()
    for _ in range(100):
        interpreter.set_tensor(input_details[0]['index'], dummy_input)
        interpreter.invoke()
    end_time = time.time()

    avg_time_ms = (end_time - start_time) / 100 * 1000
    est_fps = 1000 / avg_time_ms

    print(f"   ⚡ CPU Inference: {avg_time_ms:.2f}ms ({est_fps:.1f} FPS)")

except Exception as e:
    print(f"   ❌ TFLite speed test failed: {e}")

# Final summary
print(f"\n🎯 GROVE AI VISION DEPLOYMENT SUMMARY")
print("="*60)
print(f"{'Model':<20} {'mAP@0.5':<10} {'mAP@0.5:0.95':<12}")
print("-" * 45)

for name, metrics in results.items():
    print(f"{name:<20} {metrics['map50']:<10.4f} {metrics['map50_95']:<12.4f}")

print(f"\n📦 FINAL DEPLOYMENT MODEL:")
print(f"   📁 File: {final_tflite}")
print(f"   📏 Size: {tflite_size:.2f}MB")
print(f"   🎯 Input: 192x192x3 (INT8)")
print(f"   🚀 Optimized: QAT + Representative Dataset Calibration")
print(f"   🤖 Target: ARM Cortex-M55 + Ethos-U55 NPU")

# Performance assessment
qat_map50 = results['QAT (192)']['map50']
stage2_map50 = results['Stage 2 (192)']['map50']
retention = (qat_map50 / stage2_map50 * 100) if stage2_map50 > 0 else 0

if retention >= 95:
    print(f"   ✅ QAT Performance: EXCELLENT ({retention:.1f}% retention)")
elif retention >= 90:
    print(f"   ✅ QAT Performance: GOOD ({retention:.1f}% retention)")
elif retention >= 85:
    print(f"   ⚠️  QAT Performance: ACCEPTABLE ({retention:.1f}% retention)")
else:
    print(f"   ❌ QAT Performance: NEEDS IMPROVEMENT ({retention:.1f}% retention)")

print(f"\n🎉 PIPELINE COMPLETED SUCCESSFULLY!")
print(f"🚀 Grove AI Vision deployment model ready!")


📊 COMPREHENSIVE PIPELINE VALIDATION

🔍 Validating Stage 1 (640→192)...
Ultralytics 8.3.157 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 591.5±139.8 MB/s, size: 30.8 KB)


val: Scanning /content/head_detection_on_bus-4/valid/labels.cache... 479 images, 0 backgrounds, 0 corrupt: 100%|██████████| 479/479 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 30/30 [00:05<00:00,  5.80it/s]


                   all        479        889       0.71      0.421      0.503      0.268
Speed: 0.1ms preprocess, 2.2ms inference, 0.0ms loss, 2.4ms postprocess per image
Results saved to runs/detect/val
   mAP@0.5: 0.5031

🔍 Validating Stage 2 (192)...
Ultralytics 8.3.157 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 728.4±145.8 MB/s, size: 37.5 KB)


val: Scanning /content/head_detection_on_bus-4/valid/labels.cache... 479 images, 0 backgrounds, 0 corrupt: 100%|██████████| 479/479 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 30/30 [00:05<00:00,  5.60it/s]


                   all        479        889      0.918      0.835      0.923      0.581
Speed: 0.1ms preprocess, 1.4ms inference, 0.0ms loss, 2.3ms postprocess per image
Results saved to runs/detect/val2
   mAP@0.5: 0.9229

🔍 Validating QAT (192)...
Ultralytics 8.3.157 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 687.0±123.2 MB/s, size: 36.3 KB)


val: Scanning /content/head_detection_on_bus-4/valid/labels.cache... 479 images, 0 backgrounds, 0 corrupt: 100%|██████████| 479/479 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 30/30 [00:05<00:00,  5.48it/s]


                   all        479        889      0.942      0.833      0.927      0.588
Speed: 0.2ms preprocess, 1.7ms inference, 0.0ms loss, 2.6ms postprocess per image
Results saved to runs/detect/val3
   mAP@0.5: 0.9267

⚡ TESTING TFLITE INFERENCE SPEED
   ❌ TFLite speed test failed: Unsupported dtype dtype('float32') for randint

🎯 GROVE AI VISION DEPLOYMENT SUMMARY
Model                mAP@0.5    mAP@0.5:0.95
---------------------------------------------
Stage 1 (640→192)    0.5031     0.2676      
Stage 2 (192)        0.9229     0.5806      
QAT (192)            0.9267     0.5875      

📦 FINAL DEPLOYMENT MODEL:
   📁 File: /content/grove_ai_qat_training/grove_ai_final_deployment.tflite
   📏 Size: 2.99MB
   🎯 Input: 192x192x3 (INT8)
   🚀 Optimized: QAT + Representative Dataset Calibration
   🤖 Target: ARM Cortex-M55 + Ethos-U55 NPU
   ✅ QAT Performance: EXCELLENT (100.4% retention)

🎉 PIPELINE COMPLETED SUCCESSFULLY!
🚀 Grove AI Vision deployment model ready!
